<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

* 代码基于 GraphSAGE论文第一作者 William L. Hamilton 开源的 pytorch 实现  ： https://github.com/williamleif/graphsage-simple， 代码相对于源代码加了注释和少量改动

* 按照 William L. Hamilton 所说， 这个实现更适用于小图
    > the code is easier to read and it performs better (in terms of speed) on small-graph benchmarks.
    
* 目前只包含论文中 GraphSAGE-mean 和 GraphSAGE-GCN 两种方法

* 数据集目前只支持 cora

In [2]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict

In [3]:
class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """
    def __init__(self, features, cuda=False, gcn=False):
        """
        Initializes the aggregator for a specific graph.

        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator, self).__init__()  # 确保父类被正确的初始化了

        self.features = features
        self.cuda = cuda
        self.gcn = gcn

    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        
        每次运行时都会执行的步骤，所有自定义的module都要重写这个函数
        """

        _set = set  # Local pointers to functions (speed hack) 使用内嵌 set 函数
        if not num_sample is None:  # 如果要进行采样： 有采样数
            _sample = random.sample  # 使用 random中sample 函数 ：不放回的从一个list 或者 set 中筛选指定书目的数据
            samp_neighs = [
                _set(_sample(
                    to_neigh,
                    num_sample,
                )) if len(to_neigh) >= num_sample else to_neigh
                for to_neigh in to_neighs
            ]  # 对于所有节点， 如果邻居数大于节点数，就进行采样并转换为set， 如果不大于，就直接使用 邻居
        else:  # 如果不采样，就直接使用邻居
            samp_neighs = to_neighs

        if self.gcn:  # 采样的邻居节点中是否加入自身的节点
            samp_neighs = [
                samp_neigh.union(set([nodes[i]]))
                for i, samp_neigh in enumerate(samp_neighs)
            ]

        unique_nodes_list = list(set.union(*samp_neighs)) # 所有节点采样到的邻居节点 ： 训练使用到的节点 m
        unique_nodes = {n: i for i, n in enumerate(unique_nodes_list)} # 为用到的节点创建字典
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes))) # 创建一个矩阵 ： 所有节点 * 用到的节点： (n,m)
        # mask 矩阵 每一个节点对应的一行，  一行包含所有可能用到的节点， 是该节点用到的邻居节点位置值为1 ， 其余位置为0
        column_indices = [
            unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh
        ]  # 将邻居节点 按照创建的字典生成相应的 index
        """
        1号节点邻居对应的新编号 5,6,7,8; 2 号节点邻居对应新编号：3,4,5 --> [5,6,7,8,3,4,5] 
        for samp_neigh in samp_neighs:
            for n in samp_neigh:
                unique_nodes[n] 
        """
        row_indices = [
            i for i in range(len(samp_neighs))
            for j in range(len(samp_neighs[i]))
        ]
        
        """
        1 号节点有 4个邻居; 2号节点3个邻居， 就表示为 [1,1,1,1,2,2,2]
        for i in range(len(samp_neighs)):
            for j in range(len(samp_neighs[i])):
                i 
        """
        mask[row_indices, column_indices] = 1
        """      
        [5,6,7,8,3,4,5] 
        [1,1,1,1,2,2,2] 
        
        所以 mask 矩阵 [1,5], [1,6]...  处值为1。 表示节点 1 有新编号为 5， 6 的邻居
        """

        if self.cuda:
            mask = mask.cuda()
            
        num_neigh = mask.sum(1, keepdim=True) # torch.sum()  (n,m) --> (n, 1)
        #若keepdim值为True，则在输出张量中，除了被操作的dim维度值降为1，其它维度与输入张量input相同。
        #否则，dim维度相当于被执行torch.squeeze()维度压缩操作，导致此维度消失，最终输出张量会比输入张量少一个维度。
        mask = mask.div(num_neigh) # 按行进行归一化 [1,0,1,1,0,0] -->[1/6,0,1/6,1/6,0,0]

        if self.cuda:
            embed_matrix = self.features(
                torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list)) # (m,f) -- m: 候选节点数 ， 节点的特征数 f
        to_feats = mask.mm(embed_matrix) # 矩阵相乘 (n,m) * (m,k) --> (n,k)， 相当于进行邻居聚合： 每一行对应邻居位置的特征加和
        return to_feats

In [4]:
class Encoder(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self,
                 features,
                 feature_dim,
                 embed_dim,
                 adj_lists,
                 aggregator,
                 num_sample=10,
                 base_model=None,
                 gcn=False,
                 cuda=False,
                 feature_transform=False):
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn  # 是否拼接自身节点特征  False 是拼接
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
            torch.FloatTensor(
                embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim
            )  # 不拼接 [embed_dim,feat_dim ] ; 拼接：[embed_dim, 2*feat_dim ] 
        )  # 当Paramenters赋值给Module的属性的时候，他会自动的被加到 Module的 参数列表中
        init.xavier_uniform_(self.weight)  # 进行参数初始化 xavier 初始化

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.

        nodes     -- list of nodes
        """
        neigh_feats = self.aggregator.forward(
            nodes, [self.adj_lists[int(node)] for node in nodes],
            self.num_sample)  # (节点， 节点邻居， 采样数)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(
                    torch.LongTensor(nodes))  # (n , f) n : 节点数
            combined = torch.cat([self_feats, neigh_feats], dim=1)  # (n , 2f)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t(
        )))  # relu W* F^T : (e, 2f) * (2f, n)  --> (e,n)  e: 嵌入向量维度
        return combined

In [5]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc # 嵌入函数
        self.xent = nn.CrossEntropyLoss() # 交叉熵损失函数

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim)) # (c,e)  自动的被加到 Module的 参数列表中
        init.xavier_uniform_(self.weight) # 进行参数初始化 xavier 初始化

    def forward(self, nodes):
        # GraphSAGE 模型
        embeds = self.enc(nodes) # (e,n)  e: 嵌入向量维度
        scores = self.weight.mm(embeds) #  W* E : 嵌入结果进行一个全连接映射。 (c,e) * (e,n)  --> (c,n) c : 类别数
        return scores.t()  # (c,n) -->(n,c) 

    def loss(self, nodes, labels):
        scores = self.forward(nodes) # 分类结果
        return self.xent(scores, labels.squeeze())

In [6]:
np.empty((4,1))

array([[2.68156159e+154],
       [1.29074453e-231],
       [3.50977942e+064],
       [2.78134232e-309]])

In [13]:
def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats))  # （n,f)
    labels = np.empty((num_nodes, 1), dtype=np.int64)  # (n,1)
    node_map = {}
    label_map = {}
    with open("data/cora/cora.content") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            feat_data[i, :] = list(map(float, info[1:-1]))  # 获取节点特征
            node_map[info[0]] = i  # 节点字典
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]  # 节点标签

    adj_lists = defaultdict(set)  # 当字典里的key不存在但被查找时 , 不反回 keyerror
    with open("data/cora/cora.cites") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)  # 互相添加邻居节点
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists


def run_cora():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 2708
    feat_data, labels, adj_lists = load_cora()
    features = nn.Embedding(2708, 1433)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data),
                                   requires_grad=False)

    agg1 = MeanAggregator(features, gcn=False, cuda=False)  # 采样的邻居节点中不加入自身的节点
    enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True,
                   cuda=False)  # 不拼接自身特征
    agg2 = MeanAggregator(lambda nodes: enc1(nodes).t(),
                          cuda=False)  # 聚合第一次嵌入的特征结果， 采样的邻居节点中不加入自身的节点
    enc2 = Encoder(lambda nodes: enc1(nodes).t(),
                   enc1.embed_dim,
                   128,
                   adj_lists,
                   agg2,
                   base_model=enc1,
                   gcn=True,
                   cuda=False)  # 不拼接自身特征
    enc1.num_samples = 5  # 第一次采样邻居数
    enc2.num_samples = 5  # 第二次采样邻居数

    graphsage = SupervisedGraphSage(7, enc2)  # cora 数据集有7 个类别
    rand_indices = np.random.permutation(num_nodes)  # 随机打乱节点
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad,
                                       graphsage.parameters()),
                                lr=0.7)
    times = []
    for batch in range(100):
        """
        每个 epoch 只有 256 个数据， 没有用到所有数据？？？
        """
        batch_nodes = train[:256]  # 每个batch 256个
        random.shuffle(train)  # 打乱 train
        start_time = time.time()
        optimizer.zero_grad()  # 梯度清零
        train_output = graphsage.forward(batch_nodes)
        acc_train = f1_score(labels[batch_nodes],
                             train_output.data.numpy().argmax(axis=1),
                             average="micro") # 训练集准确率
        loss = graphsage.loss(
            batch_nodes,
            Variable(torch.LongTensor(labels[np.array(batch_nodes)])))  # 计算训练损失
        loss.backward()  # 损失梯度计算，回传
        optimizer.step()  #  参数更新
        end_time = time.time()
        times.append(end_time - start_time)

        val_output = graphsage.forward(val)  # 没有使用 eval() 因为没有 BN 和dropout
        loss_val = graphsage.loss(
            val, Variable(torch.LongTensor(labels[np.array(val)])))  # 计算损失
        acc_val = f1_score(labels[val],
                           val_output.data.numpy().argmax(axis=1),
                           average="micro")

        print(
            'Epoch: {:04d}'.format(batch + 1),
            'loss_train: {:.4f}'.format(loss.item()),
            'acc_train: {:.4f}'.format(acc_train.item()),
            'loss_val: {:.4f}'.format(loss_val.item()),
            'acc_val: {:.4f}'.format(acc_val.item()),
            'time: {:.4f}s'.format(end_time - start_time)
        )

    print("Average batch time:", np.mean(times))
    
    
    # 测试部分
    test_output = graphsage.forward(test)  # 没有使用 eval() 没有 BN 和dropout
    loss_test = graphsage.loss(
        test, Variable(torch.LongTensor(labels[np.array(test)])))  # 计算损失
    acc_test = f1_score(labels[test],
                        test_output.data.numpy().argmax(axis=1),
                        average="micro")
    print("Test set results:", "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [14]:
run_cora()

Epoch: 0001 loss_train: 1.9394 acc_train: 0.1758 loss_val: 1.9299 acc_val: 0.1940 time: 0.0741s
Epoch: 0002 loss_train: 1.9258 acc_train: 0.2578 loss_val: 1.9065 acc_val: 0.3760 time: 0.0650s
Epoch: 0003 loss_train: 1.9037 acc_train: 0.3594 loss_val: 1.8806 acc_val: 0.3940 time: 0.0710s
Epoch: 0004 loss_train: 1.8642 acc_train: 0.4102 loss_val: 1.8514 acc_val: 0.4060 time: 0.0792s
Epoch: 0005 loss_train: 1.8409 acc_train: 0.4258 loss_val: 1.8118 acc_val: 0.3740 time: 0.0702s
Epoch: 0006 loss_train: 1.7966 acc_train: 0.3828 loss_val: 1.7648 acc_val: 0.3500 time: 0.0721s
Epoch: 0007 loss_train: 1.7636 acc_train: 0.3203 loss_val: 1.7157 acc_val: 0.3540 time: 0.0689s
Epoch: 0008 loss_train: 1.6945 acc_train: 0.3984 loss_val: 1.6628 acc_val: 0.3700 time: 0.0724s
Epoch: 0009 loss_train: 1.6266 acc_train: 0.4180 loss_val: 1.6142 acc_val: 0.3560 time: 0.0653s
Epoch: 0010 loss_train: 1.5626 acc_train: 0.3516 loss_val: 1.5650 acc_val: 0.4180 time: 0.0639s
Epoch: 0011 loss_train: 1.5606 acc_train

Epoch: 0087 loss_train: 0.1906 acc_train: 0.9414 loss_val: 0.4618 acc_val: 0.8660 time: 0.0711s
Epoch: 0088 loss_train: 0.1689 acc_train: 0.9336 loss_val: 0.4753 acc_val: 0.8460 time: 0.0869s
Epoch: 0089 loss_train: 0.1877 acc_train: 0.9297 loss_val: 0.4414 acc_val: 0.8720 time: 0.0638s
Epoch: 0090 loss_train: 0.2151 acc_train: 0.9258 loss_val: 0.4465 acc_val: 0.8660 time: 0.0676s
Epoch: 0091 loss_train: 0.2066 acc_train: 0.9453 loss_val: 0.4788 acc_val: 0.8520 time: 0.0720s
Epoch: 0092 loss_train: 0.2375 acc_train: 0.9492 loss_val: 0.4482 acc_val: 0.8600 time: 0.0706s
Epoch: 0093 loss_train: 0.1756 acc_train: 0.9375 loss_val: 0.4505 acc_val: 0.8680 time: 0.1150s
Epoch: 0094 loss_train: 0.1968 acc_train: 0.9453 loss_val: 0.5029 acc_val: 0.8420 time: 0.0776s
Epoch: 0095 loss_train: 0.2740 acc_train: 0.9141 loss_val: 0.4688 acc_val: 0.8620 time: 0.0701s
Epoch: 0096 loss_train: 0.1604 acc_train: 0.9375 loss_val: 0.4615 acc_val: 0.8520 time: 0.0801s
Epoch: 0097 loss_train: 0.1679 acc_train